In [1]:
# PROBE_CHARACTERISATION.IPYNB - A showcase of how the CBC probe module can be characterised using constitutive reporters
# By Kirill Sechkar

In [2]:
# PACKAGE IMPORTS
import numpy as np
import jax
import jax.numpy as jnp
import functools
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB

import time

# CIRCUIT AND EXTERNAL INPUT IMPORTS
from sim_tools.cell_model import *
import sim_tools.genetic_modules as gms
import sim_tools.controllers as ctrls
import sim_tools.reference_switchers as refsws
import sim_tools.ode_solvers as odesols
from probe_char_tools import *

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [3]:
# PART 1: CHARACTERISE CONSTITUTIVE REPORTERS

# DEFINE CONSTITUTIVE REPORTER PARAMETERS
# for simplicity, just consider different promoter strengths for same copy numbers and RBSs
# assuming protein maturation rates known (andidentical for simplicity)

# default parameteres for all constitutive reporters
constrep_default_par={}
constrep_default_par['func_'] = 1.0  # gene functionality - 1 if working, 0 if mutated
constrep_default_par['c_'] = 10.0  # copy no. (nM)
constrep_default_par['a_'] = 100.0  # promoter strength (unitless)
constrep_default_par['b_'] = 6.0  # mRNA decay rate (/h)
constrep_default_par['k+_'] = 60.0  # ribosome binding rate (/h/nM)
constrep_default_par['k-_'] = 60.0  # ribosome unbinding rate (/h)
constrep_default_par['n_'] = 300.0  # protein length (aa)
constrep_default_par['d_'] = 0.0  # rate of active protein degradation by synthetic protease - zero by default (/h/nM)
# default maturation rate
constrep_default_par['mu_ofp2']=1/(13.6/60)   # sfGFP maturation time of 13.6 min

# define promoter strengths considered
# feasible range between 40 and 2500 (Sechkar et al., Nature Comms 2024)
# you don't have to, but order of increasing magnitudes will make things look neat
constrep_as=[50,2000]

# define maturation rates considered
constrep_mus = [1/(13.6/60), 1/(13.6/60)]  # sfGFP maturation time of 13.6 min

# define reporter pairs to consider
constrep_pairs = [(0,1)]

In [4]:
# SET SIMULATION AND CONTROLLER PARAMETERS

# set controller parameters - irrelevant for the constitutive reporter characterisation
refs = jnp.array([0.0])
control_delay = 0  # control action delay
u0 = 0.0  # initial control action

 # set simulation parameters
tf = (0.0, 48.0)  # simulation time frame - assuming the steady state is reached in 50 hours

# measurement time step
meastimestep = 48.0  # hours - we only care about the steady state at the end

# choose Euler integration timestep
euler_timestep = 1e-5  # hours

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [5]:
# INITIALISE AND PARAMETERISE THE CELL MODEL AND ODE SOLVER

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
cellmodel_par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(cellmodel_par)  # get default initial conditions

# add reference tracker switcher
cellmodel_par_with_refswitch, ref_switcher = cellmodel_auxil.add_reference_switcher(cellmodel_par,  # cell model parameters
                                                                                    refsws.no_switching_initialise, # function initialising the reference switcher (none here)
                                                                                    refsws.no_switching_switch  # function switching the references to be tracked (none here)
                                                                                    )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_ss_and_each, synth_miscs_total_ss_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = cellmodel_auxil.add_modules_and_controller(
        # module 1 - one constitutive reporter
        gms.constfp_initialise,  # function initialising the circuit
        gms.constfp_ode,  # function defining the circuit ODEs
        gms.constfp_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.constfp_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # module 2 - another constitutive reporter
        gms.constfp2_initialise,  # function initialising the circuit
        gms.constfp2_ode,  # function defining the circuit ODEs
        gms.constfp2_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.constfp2_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # controller
        ctrls.cci_initialise,  # function initialising the controller
        ctrls.cci_action,  # function calculating the controller action
        ctrls.cci_ode,  # function defining the controller ODEs
        ctrls.cci_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        cellmodel_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_ss_and_each[0]
module1_genes = synth_genes_total_ss_and_each[1]
module2_genes = synth_genes_total_ss_and_each[2]
synth_miscs = synth_miscs_total_ss_and_each[0]
module1_miscs = synth_miscs_total_ss_and_each[1]
module2_miscs = synth_miscs_total_ss_and_each[2]

# set constitutive reporter parameters to default values
for gene in synth_genes:
    for key in constrep_default_par.keys():
        cellmodel_par[key+gene] = constrep_default_par[key]

ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=euler_timestep)

In [6]:
# SIMULATE MEASUREMENTS FOR INDIVIDUAL REPORTERS
mature_ofp_ss_individual = np.zeros(len(constrep_as)) # initialise storage for steady-state mature protein levels
l_ss_individual = np.zeros(len(constrep_as))   # initialise storage for steady-state cell growth rates
for i in range(0,len(constrep_as)):
    # set constitutive reporter parameters depending on which reporter is considered
    cellmodel_par['a_ofp'] = constrep_as[i]
    cellmodel_par['a_ofp2'] = 0 #   no second reporter in the cell

    # solve ODE
    ts_jnp, xs_jnp,\
    ctrl_ss_memorecord_jnp, uexprecord_jnp, \
    refrecord_jnp  = ode_sim(par,   # model parameters
                             ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                             odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                             controller_ctrledvar,    # name of the variable read and steered by the controller
                             controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                             cellmodel_auxil.x0_from_init_conds(init_conds,
                                                                par,
                                                                synth_genes, synth_miscs, controller_dynvars,
                                                                modules_name2pos,
                                                                controller_name2pos),   # initial condition VECTOR
                             controller_memo0,  # initial controller memory record
                             u0,    # initial control action, applied before any measurement-informed actions reach the sy_probestem
                             (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                             (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                             modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                             cellmodel_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                             tf, meastimestep,  # simulation time frame and measurement time step
                             control_delay,  # delay before control action reaches the sy_probestem
                             us_size,  # size of the control action record needed
                             refs, ref_switcher,  # reference values and reference switcher
                             )
    # convert simulation results to numpy array_probes
    ts = np.array(ts_jnp)
    xs = np.array(xs_jnp)

    # record mature protein levels
    mature_ofp_ss_individual[i] = xs[-1,modules_name2pos['ofp_mature']]

    # calculate and record growth rates
    _, ls_jnp, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D(ts, xs, par,
                                                                    synth_genes, synth_miscs,
                                                                    modules_name2pos,
                                                                    module1_specterms, module2_specterms)
    l_ss_individual[i] = np.array(ls_jnp)[-1]

In [7]:
# SIMULATE MEASUREMENTS FOR REPORTER PAIRS

mature_ofp_ss_pairs = {} # initialise storage for steady state mature protein levels
l_ss_pairs = {}   # initialise storage for growth rates

# simulate measurements for each reporter pair
for i in range(0,len(constrep_pairs)):
    constrep_pair = constrep_pairs[i]

    # set constitutive reporter parameters depending on which pair is considered
    cellmodel_par['a_'+synth_genes[constrep_pair[0]]] = constrep_as[constrep_pair[0]]
    cellmodel_par['a_'+synth_genes[constrep_pair[1]]] = constrep_as[constrep_pair[1]]

    # solve ODE
    ts_jnp, xs_jnp,\
    ctrl_ss_memorecord_jnp, uexprecord_jnp, \
    refrecord_jnp  = ode_sim(par,   # model parameters
                             ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                             odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                             controller_ctrledvar,    # name of the variable read and steered by the controller
                             controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                             cellmodel_auxil.x0_from_init_conds(init_conds,
                                                                par,
                                                                synth_genes, synth_miscs, controller_dynvars,
                                                                modules_name2pos,
                                                                controller_name2pos),   # initial condition VECTOR
                             controller_memo0,  # initial controller memory record
                             u0,    # initial control action, applied before any measurement-informed actions reach the sy_probestem
                             (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                             (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                             modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                             cellmodel_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                             tf, meastimestep,  # simulation time frame and measurement time step
                             control_delay,  # delay before control action reaches the sy_probestem
                             us_size,  # size of the control action record needed
                             refs, ref_switcher,  # reference values and reference switcher
                             )
    # convert simulation results to numpy array_probes
    ts = np.array(ts_jnp)
    xs = np.array(xs_jnp)

    # record abolsute mature protein levels for the pair
    mature_ofp_ss_pairs[constrep_pair] = {}
    mature_ofp_ss_pairs[constrep_pair][constrep_pair[0]] = xs[-1,modules_name2pos['ofp_mature']]
    mature_ofp_ss_pairs[constrep_pair][constrep_pair[1]] = xs[-1,modules_name2pos['ofp2_mature']]

    # calculate and record growth rate for the pair
    _, ls_jnp, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D(ts, xs, par,
                                                                    synth_genes, synth_miscs,
                                                                    modules_name2pos,
                                                                    module1_specterms, module2_specterms)
    l_ss_pairs[constrep_pair] = np.array(ls_jnp)[-1]

In [8]:
# CALCULATE RESOURCE COMPETITION FACTORS FOR THE REPORTERS
Q_constrep = -np.ones(len(constrep_as)) # initialise resource competition factors array with impossible -1 values

# calculate Q values for the reporters
for i in range(0,len(constrep_pairs)):
    constrep_pair = constrep_pairs[i]

    # get the Q values
    Q0, Q1 = Q_calc_both(constrep_mus[constrep_pair[0]], constrep_mus[constrep_pair[1]],                # maturation rates of the first and second output fluorescent proteins
                         mature_ofp_ss_individual[constrep_pair[0]], l_ss_individual[constrep_pair[0]], # individual measurements for the first module: output prot. conc., growth rate
                         mature_ofp_ss_individual[constrep_pair[1]], l_ss_individual[constrep_pair[1]], # individual measurements for the second module: output prot. conc., growth rate
                         mature_ofp_ss_pairs[constrep_pair][constrep_pair[0]],
                         mature_ofp_ss_pairs[constrep_pair][constrep_pair[1]], l_ss_pairs[constrep_pair]    # pair measurements (this and above lines): output prot. concs., growth rate
                         )

    # get Q for the first reporter in the pair - if not already found from measurements for another pair
    if (Q_constrep[constrep_pair[0]] == -1):
        Q_constrep[constrep_pair[0]] = Q0


    # get Q for the second reporter in the pair - if not already found from measurements for another pair
    if (Q_constrep[constrep_pair[1]] == -1):
        Q_constrep[constrep_pair[1]] = Q1

In [9]:
# PART 2: CHARACTERISE THE PROBE

# DEFINE PROBE PARAMETERS
probe_par = {}
probe_par['c_ta'] = 100.0
probe_par['a_ta'] = 10.0
probe_par['c_b'] = 100.0
probe_par['a_b'] = 5000.0
probe_par['mu_b'] = 1/(13.6/60)

# define inducer levels to consider
# you don't have to, but order of increasing magnitudes will make things look neat
us=[0, 1, 10, 100, 1000]

In [10]:
# SET SIMULATION AND CONTROLLER PARAMETERS

# set controller parameters - constant chemical inducer level
refs = jnp.array([0.0]) # control references - irrelevant for constant control input
control_delay = 0  # control action delay - irrelevant for constant control input
u0 = 0.0  # initial control action - just a default that we'll adjust based on the input considered

 # set simulation parameters
tf = (0.0, 48.0)  # simulation time frame - assuming the steady state is reached in 50 hours

# measurement time step
meastimestep = 48.0  # hours - we only care about the steady state at the end

In [11]:
# INITIALISE AND PARAMETERISE THE CELL MODEL

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
cellmodel_par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(cellmodel_par)  # get default initial conditions

# add reference tracker switcher
cellmodel_par_with_refswitch, ref_switcher = cellmodel_auxil.add_reference_switcher(cellmodel_par,  # cell model parameters
                                                                                    refsws.no_switching_initialise, # function initialising the reference switcher (none here)
                                                                                    refsws.no_switching_switch  # function switching the references to be tracked (none here)
                                                                                    )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_ss_and_each, synth_miscs_total_ss_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = cellmodel_auxil.add_modules_and_controller(
        # module 1 - one constitutive reporter
        gms.constfp_initialise,  # function initialising the circuit
        gms.constfp_ode,  # function defining the circuit ODEs
        gms.constfp_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.constfp_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # module 2 - the probe
        gms.cicc_initialise,  # function initialising the circuit
        gms.cicc_ode,  # function defining the circuit ODEs
        gms.cicc_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.cicc_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # controller
        ctrls.cci_initialise,  # function initialising the controller
        ctrls.cci_action,  # function calculating the controller action
        ctrls.cci_ode,  # function defining the controller ODEs
        ctrls.cci_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        cellmodel_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_ss_and_each[0]
module1_genes = synth_genes_total_ss_and_each[1]
module2_genes = synth_genes_total_ss_and_each[2]
synth_miscs = synth_miscs_total_ss_and_each[0]
module1_miscs = synth_miscs_total_ss_and_each[1]
module2_miscs = synth_miscs_total_ss_and_each[2]

# set constitutive reporter parameters to default values
for gene in synth_genes:
    for key in constrep_default_par.keys():
        cellmodel_par[key+gene] = constrep_default_par[key]

# set up the ODE solver
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=euler_timestep)

In [31]:
# SIMULATE MEASUREMENTS OF THE PROBE'S STEADY STATE WITH DIFFERENT REPORTERS AND INPUTS

# add the 'no constitutive reporter case' by inserting zero transcription rate at the constitutive reporter list's beginning (if needed)
if(0 not in constrep_as):
    constrep_as.insert(0, 0.0)
    constrep_mus.insert(0, 0.0)

# initialise storage - us on axis 0, Q_constreps on axis 1
l_ss = -np.ones((len(us),len(constrep_as)))         # cell growth rates
y_probes = -np.ones((len(us),len(constrep_as)))           # probe outputs, i.e. mature probe output protein levels (alias mature_b_ss)
Q_probes = -np.ones((len(us),len(constrep_as)))     # calculated normalised resource competition factors

# simulate measurements for each inducer level and constitutive reporter combination
for i in range(0,len(us)):
    for j in range(0, len(constrep_as)):
        # set the inducer level for the next simulationm
        init_conds['inducer_level'] = us[i]

        # set the constitutive reporter parameter for the next simulation
        cellmodel_par['a_ofp'] = constrep_as[j]
        
        # solve ODE
        ts_jnp, xs_jnp,\
        ctrl_memorecord_jnp, uexprecord_jnp, \
        refrecord_jnp  = ode_sim(par,   # model parameters
                                 ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                                 odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                                 controller_ctrledvar,    # name of the variable read and steered by the controller
                                 controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                                 cellmodel_auxil.x0_from_init_conds(init_conds,
                                                                    par,
                                                                    synth_genes, synth_miscs, controller_dynvars,
                                                                    modules_name2pos,
                                                                    controller_name2pos),   # initial condition VECTOR
                                 controller_memo0,  # initial controller memory record
                                 u0,    # initial control action, applied before any measurement-informed actions reach the sy_probestem
                                 (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                                 (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                                 modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                                 cellmodel_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                                 tf, meastimestep,  # simulation time frame and measurement time step
                                 control_delay,  # delay before control action reaches the sy_probestem
                                 us_size,  # size of the control action record needed
                                 refs, ref_switcher,  # reference values and reference switcher
                                 )
        # convert simulation results to numpy array_probes
        ts = np.array(ts_jnp)
        xs = np.array(xs_jnp)
        
        # record probe output
        y_probes[i,j] = xs[-1,modules_name2pos['b_mature']]
        
        # record growth rate
        _, ls_jnp, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D(ts, xs, par,
                                                                        synth_genes, synth_miscs,
                                                                        modules_name2pos,
                                                                        module1_specterms, module2_specterms)
        l = np.array(ls_jnp)[-1]
        l_ss[i, j] = l

        # record the probe's normalised resource competition factor
        if(constrep_as[j] == 0.0):  # if there is no reporter from whose output it can be deduced, skip this step
            continue
        Q_probes[i, j] = Q_calc_one(constrep_mus[j-1],                                      # maturation rate of the constitutive reporter's output fluorescent protein
                                    mature_ofp_ss_individual[j-1], l_ss_individual[j-1],    # individual measurements for the const. reporter: output prot. conc., growth rate
                                    xs[-1, modules_name2pos['ofp_mature']], l,              # pair measurements for the const. reporter: output prot. concs., growth rate
                                    Q_constrep[j-1]                                         # normalised resource competition factor for the constitutive reporter
                                    )   # using j-1 as the constitutive reporter list has an extra zero at the beginning

0 0.0
0.0
0 50
0.0
0 2000
0.0
1 0.0
1.0
1 50
1.0
1 2000
1.0
10 0.0
10.0
10 50
10.0
10 2000
10.0
100 0.0
100.0
100 50
100.0
100 2000
100.0
1000 0.0
1000.0
1000 50
1000.0
1000 2000
1000.0


[[-1.          0.00277856  0.0015959 ]
 [-1.          0.00289577  0.00165971]
 [-1.          0.0046459   0.00265634]
 [-1.          0.00503837  0.00289173]
 [-1.          0.00504976  0.00289863]]
